In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
278,AFG,Asia,Afghanistan,2020-10-04,39297.0,7.0,10.000,1462.0,4.0,1.286,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
488,ALB,Europe,Albania,2020-10-04,14117.0,152.0,137.714,392.0,3.0,2.429,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
767,DZA,Africa,Algeria,2020-10-04,51995.0,148.0,154.429,1756.0,7.0,6.429,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-03,France,602811.0,0.0,32217.0,0.0,0.0,0.0
0,2020-10-03,China,90610.0,0.0,4744.0,0.0,0.0,0.0
0,2020-10-03,Italy,322026.0,0.0,35978.0,0.0,0.0,0.0
0,2020-10-03,Spain,800419.0,0.0,32238.0,0.0,0.0,0.0
0,2020-10-03,United States,7400399.0,0.0,209819.0,0.0,0.0,0.0
0,2020-10-03,World,35167449.0,0.0,1037296.0,0.0,0.0,0.0
0,2020-10-03,United Kingdom,472519.0,0.0,42341.0,0.0,0.0,0.0
0,2020-10-03,Germany,299192.0,0.0,9538.0,0.0,0.0,0.0
0,2020-10-03,Iran,467659.0,0.0,26785.0,0.0,0.0,0.0
0,2020-10-03,Turkey,322659.0,0.0,8381.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-04,France,602811.0,606625.0,32217.0,32198.0,-3814.0,19.0
0,2020-10-04,China,90610.0,91095.0,4744.0,4739.0,-485.0,5.0
0,2020-10-04,Italy,322026.0,322751.0,35978.0,35968.0,-725.0,10.0
0,2020-10-03,Spain,800419.0,789932.0,32238.0,32086.0,10487.0,152.0
0,2020-10-04,United States,7400399.0,7382944.0,209819.0,209394.0,17455.0,425.0
0,2020-10-04,World,35167449.0,34986502.0,1037296.0,1034240.0,180947.0,3056.0
0,2020-10-04,United Kingdom,472519.0,480017.0,42341.0,42317.0,-7498.0,24.0
0,2020-10-04,Germany,299192.0,299237.0,9538.0,9529.0,-45.0,9.0
0,2020-10-04,Iran,467659.0,468119.0,26785.0,26746.0,-460.0,39.0
0,2020-10-04,Turkey,322659.0,323014.0,8381.0,8384.0,-355.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")